# `nnx` using `vmap` to create multiple models

In case of `linen`, you can you use the following initialaztion of several instaces:
```python
def make(rng):
  m = my_module.init(rng, dummy_input)
  return ...

rngs = jax.random.split(jax.random.PRNGKey(0), num=5)
models = jax.vmap(make)(rngs)
```

You can use nnx.split_rngs to automatically split the Rngs before going into nnx.vmap.

```python
@nnx.split_rngs(splits=5)
@nnx.vmap
def make_model(rngs):
  return nnx.Linear(2, 3, rngs=rngs)

model = make_model(nnx.Rngs(0))

print(model)
```

In [44]:
import jax
from flax import nnx
import jax.numpy as jnp

In [45]:
@nnx.split_rngs(splits=5)
@nnx.vmap
def make_model(rngs):
  return nnx.Linear(2, 3, rngs=rngs)

model = make_model(nnx.Rngs(0))

print(model)

Linear( # Param: 45 (180 B)
  bias=Param( # 15 (60 B)
    value=Array(shape=(5, 3), dtype=dtype('float32'))
  ),
  bias_init=<function zeros at 0x7b4862d41080>,
  dot_general=<function dot_general at 0x7b486374cd60>,
  dtype=None,
  in_features=2,
  kernel=Param( # 30 (120 B)
    value=Array(shape=(5, 2, 3), dtype=dtype('float32'))
  ),
  kernel_init=<function variance_scaling.<locals>.init at 0x7b48620c09a0>,
  out_features=3,
  param_dtype=float32,
  precision=None,
  promote_dtype=<function promote_dtype at 0x7b486206fd80>,
  use_bias=True
)


In [47]:
print("Model parameter shapes:")
print(f"Kernel shape: {model.kernel.value.shape}")
print(f"Bias shape: {model.bias.value.shape}\n")

Model parameter shapes:
Kernel shape: (5, 2, 3)
Bias shape: (5, 3)



In [56]:
num_devices = 1
per_device_batch = 4
num_heads = 2
din = 16
dout = 32

@nnx.split_rngs(splits=num_devices)
@nnx.pmap(in_axes=0, out_axes=0)  # device dim
@nnx.split_rngs(splits=per_device_batch)
@nnx.vmap(in_axes=0, out_axes=0)  # batch dim
@nnx.split_rngs(splits=num_heads)
@nnx.vmap(in_axes=0, out_axes=1)  # head dim
def make_model(rngs):
  return nnx.Linear(din, dout, rngs=rngs)

model = make_model(nnx.Rngs(0))
print(model)

Linear( # Param: 4,352 (17.4 KB)
  bias=Param( # 256 (1.0 KB)
    value=Array(shape=(1, 4, 32, 2), dtype=dtype('float32'))
  ),
  bias_init=<function zeros at 0x7b4862d41080>,
  dot_general=<function dot_general at 0x7b486374cd60>,
  dtype=None,
  in_features=16,
  kernel=Param( # 4,096 (16.4 KB)
    value=Array(shape=(1, 4, 16, 2, 32), dtype=dtype('float32'))
  ),
  kernel_init=<function variance_scaling.<locals>.init at 0x7b48620c09a0>,
  out_features=32,
  param_dtype=float32,
  precision=None,
  promote_dtype=<function promote_dtype at 0x7b486206fd80>,
  use_bias=True
)


In [57]:
print("Model parameter shapes:")
print(f"Kernel shape: {model.kernel.value.shape}")
print(f"Bias shape: {model.bias.value.shape}\n")

Model parameter shapes:
Kernel shape: (1, 4, 16, 2, 32)
Bias shape: (1, 4, 32, 2)



Notice vectorized number of paramers for two model above 45 vs 4,352 parameters. It is important what do you want exactly to vectorize. Maybe just shared parameters for each GPU.

In [50]:
# Configuration
batch = 2
num_heads = 3
head_dim = 4  # input dimension per head
dout = 5      # output dimension
seq_len = 6

# Initialize vectorized model
@nnx.split_rngs(splits=batch)
@nnx.vmap(in_axes=0)  # batch dimension
@nnx.split_rngs(splits=num_heads)
@nnx.vmap(in_axes=0)  # head dimension
def make_model(rngs):
    # Each Linear layer processes head_dim -> dout
    return nnx.Linear(head_dim, dout, rngs=rngs)

model = make_model(nnx.Rngs(0))

# Create sample input (batch, num_heads, seq_len, head_dim)
x = jnp.ones((batch, num_heads, seq_len, head_dim))

# Forward pass with vmap
def forward(model, x):
    @nnx.vmap(in_axes=(0, 0))  # batch dimension
    @nnx.vmap(in_axes=(0, 0))  # head dimension
    def forward_single(model, x):
        # x shape: (seq_len, head_dim)
        # model.kernel shape: (head_dim, dout)
        return model(x)

    return forward_single(model, x)

# Run and print shapes
print("Model parameter shapes:")
print(f"Kernel shape: {model.kernel.value.shape}")  # Should be (batch, num_heads, head_dim, dout)
print(f"Bias shape: {model.bias.value.shape}\n")   # Should be (batch, num_heads, dout)

print("Input/Output shapes:")
print(f"Input shape: {x.shape}")
output = forward(model, x)
print(f"Output shape: {output.shape}")

Model parameter shapes:
Kernel shape: (2, 3, 4, 5)
Bias shape: (2, 3, 5)

Input/Output shapes:
Input shape: (2, 3, 6, 4)
Output shape: (2, 3, 6, 5)
